In [1]:
import pandas as pd
import re
import json
import requests
from urllib.request import urlopen

filename = input("filename: ")
df = pd.read_csv(filename + '.csv')

latitudes = []
longitudes = []
counter = 0
      

def get_coords(url):
    
    try:
        coord_data = requests.get(url).json()
    except:
        print("GET query error")
        return []
    #coord_data = json.loads(urlopen(url).read().decode('utf-8'))
    if coord_data['features'] == []:
        print("Features empty")
        return []
    else:
        try:
            latitude = coord_data['features'][0]['properties']['latitude']
            longitude = coord_data['features'][0]['properties']['longitude']
            print(latitude + ', ' + longitude)
            return [latitude, longitude]
        except:
            print("No longitude/latitude field")
            return []

# iterate through rows
for index, row in df.iterrows():
    # isolate placenames column
    placenames = df.iloc[index]['placenames']
    # if the placenames column is empty, go to next row
    if placenames != "Empty" and not pd.isnull(placenames) and placenames != "" and placenames != "None":
        # split the placenames column into a list
        placenames_list = re.split(', ', placenames)
        # iterate through places
        for place in placenames_list:
            if place != "":
                url = "http://tlcmap.org/ghap/search?fuzzyname=" + place + "&format=json"
                #print(url)
                print(str(df.iloc[index]['line_num']) + ': ' + url)
                coords = get_coords(url)
                if coords == []:
                    continue
                else:
                    counter +=1
                    latitudes.append(coords[0])
                    longitudes.append(coords[1])
                    break
        # if loop iterates through all places and finds no coordinates
        counter += 1
        latitudes.append('Empty')
        longitudes.append('Empty')

coords_d = d = {'latitude': latitudes, 'longitude': longitudes}
coords_df = pd.DataFrame(data=coords_d)
final_df = pd.concat([df, coords_df], axis=1)
final_df.to_csv(filename + '_w_coords.csv', index=False)

0: http://tlcmap.org/ghap/search?fuzzyname=IBIS&format=json
-23.070555555555554, 145.05333333333334
1: http://tlcmap.org/ghap/search?fuzzyname=LAW&format=json
-24.7, 145.3
2: http://tlcmap.org/ghap/search?fuzzyname=Hun&format=json
-35.42061, 139.75712
3: http://tlcmap.org/ghap/search?fuzzyname=CASTLE&format=json
-33.70666666666667, 151.01777777777778
5: http://tlcmap.org/ghap/search?fuzzyname=BOSTON&format=json
-34.6911167, 135.8349681
6: http://tlcmap.org/ghap/search?fuzzyname=HSASIONS&format=json
-37.906511, 145.142166
7: http://tlcmap.org/ghap/search?fuzzyname=XIRLS&format=json
Features empty
7: http://tlcmap.org/ghap/search?fuzzyname=ROMISH&format=json
-32.33044, 116.60621
9: http://tlcmap.org/ghap/search?fuzzyname=WELLINGTON&format=json
-32.55013889, 149.0344444
10: http://tlcmap.org/ghap/search?fuzzyname=Roval&format=json
-34.58275, 138.92769
11: http://tlcmap.org/ghap/search?fuzzyname=Meleiina&format=json
-29.18166666666667, 149.9011111111111
12: http://tlcmap.org/ghap/search?fu